In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [3]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedKFold, StratifiedShuffleSplit, RepeatedStratifiedKFold, validation_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer, f1_score, classification_report, confusion_matrix, roc_curve, precision_recall_curve, ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay

### Область работы 2 (выполнение лучшей модели)

In [6]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [7]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [8]:
df_test=pd.read_csv(path_test)
df_train=pd.read_csv(path_train)

mask=df_test.notna().all(axis=1)
df_itog=df_test[mask].copy()

df_train=df_train.dropna()
df_train = df_train.drop_duplicates()

df_train['qual'] = np.where(df_train['quality'] >= 7, 1, 0)
df_train = df_train.drop('quality', axis=1)
df_train.rename(columns={'qual': 'quality'}, inplace=True)

X = df_train.drop('quality', axis=1)
y = df_train['quality']

gnb = make_pipeline(RobustScaler(), LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced', max_iter=10000))
gnb.fit(X, y)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', max_iter=10000,
                                    solver='liblinear'))])

In [9]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
threshold = 0.914
y_probs = gnb.predict_proba(df_itog)[:, 1]  
y_predict = (y_probs >= threshold).astype(int)